In [1]:
import sys
sys.path.append('../')
from Base import Scraper

import pandas as pd
import re

In [2]:
p = re.compile('(\d{3,3}-\d{3,3}-\d{4,4})')

scraper = Scraper()

names = []
companies = []
streets = []
cities = []
states = []
zips = []
countries = []
languages_spoken = []
links_to_page = []
phone_numbers = []

for page_n in range(1, 15):
    link_to_page = []
    print('Page number:', page_n)
    scraper.navigate(f'https://www.century21global.com/real-estate-agents/USA;jsessionid=C09FE3FEB25CDC1526A725D497BEF710.worker1?pageNo={page_n}')
    
    people = scraper.driver.find_elements_by_xpath("/html/body/div[2]/div[3]/div/div[3]/div/div/div[2]/div[starts-with(@class, 'search')]")
    print('Number of agents found:', len(people))
    for person in people:
        try:
            names.append(person.find_element_by_xpath(".//span[contains(@class, 'name-label')]").text)
            print(names[-1])
        except:
            names.append(None)
            print('Name not found')
        try:
            companies.append(person.find_element_by_xpath(".//span[contains(@class, 'label-secondary')]").text)
        except:
            companies.append(None)
            print('Company not found')
        try:
            address_lang = person.find_elements_by_xpath(".//span[@class='search-result-label']")
            street_city, state_country = address_lang[0].text.split('\n')
            street, city = street_city.split(',')
            state_zip, country = state_country.split(',')
            zip_ = state_zip[-6:].strip()
            state = state_zip[:-6].strip()
            
            streets.append(street)
            cities.append(city)
            states.append(state)
            zips.append(zip_)
            countries.append(country)
            
            
            if len(address_lang) == 2:
                language_spoken = address_lang[1].text.split(':')[-1].strip()
                languages_spoken.append(language_spoken)
            else:
                languages_spoken.append(None)
        except:
            streets.append(None)
            cities.append(None)
            states.append(None)
            zips.append(None)
            countries.append(None)
            languages_spoken.append(None)
            
            print('No address info')
        try:
            link_to_page.append(person.find_element_by_xpath(".//a[@class='search-result-photo result-photo-portrait']").get_attribute('href'))
        except:
            print('No link')
            link_to_page.append(None)
    links_to_page += link_to_page
    for n, link in enumerate(link_to_page):
        try:
            scraper.navigate(link)
            find_number = scraper.driver.find_element_by_xpath("//div[@class='detail-data-primary']").text.split('\n')
            for pat in find_number:
                if 'phone:' in pat.lower():
                    phone_numbers.append(pat.split(' ')[-1])
                    break
            else:
                print('No number found in description:', n)
                phone_numbers.append(None)
        except:
            print('No number found')
            phone_numbers.append(None)
    print('-' * 100)

OSError: [Errno 8] Exec format error: '../../chromedriver'

In [68]:
pd_dict = dict()
pd_dict['Name'] = names
pd_dict['Agency'] = companies
pd_dict['Language Spoken'] = languages_spoken
pd_dict['Phone Number'] = phone_numbers
pd_dict['Street'] = streets
pd_dict['City'] = cities
pd_dict['State'] = states
pd_dict['Zip'] = zips
pd_dict['County'] = countries
pd_dict['Link to page'] = links_to_page

df = pd.DataFrame(pd_dict)
print(df.head())
df.to_csv('real_estate_agents.csv', index=False)

                                                Name  \
0                                     Bradley Realty   
1                      Asa Cox of Asa Cox Homes Team   
2                                    Michael Kenduck   
3  Linda Frierdich of The Linda Frierdich Real Es...   
4                         David Parker of Elite Team   

                                   Agency Language Spoken Phone Number  \
0         CENTURY 21 Bradley Realty, Inc.         English   2603991177   
1                CENTURY 21 Asa Cox Homes         English         None   
2               CENTURY 21 American Homes            None   6318972578   
3  CENTURY 21 Advantage Real Estate, Inc.         English         None   
4                      CENTURY 21 Everest            None         None   

               Street          City     State    Zip County  \
0  2928 E. Dupont Rd.    Fort Wayne   Indiana  46825    USA   
1  2709 N. Ridge Road   Painesville      Ohio  44077    USA   
2  40 Underhill Blvd.       S